In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image

In [22]:
PIC_DIR = './data/img_align_celeba/img_align_celeba/'

In [23]:
IMAGES_COUNT = 10000

In [24]:
ORIG_WIDTH = 178
ORIG_HEIGHT = 208
diff = (ORIG_HEIGHT - ORIG_WIDTH) // 2

In [25]:
WIDTH = 128
HEIGHT = 128

In [26]:
crop_rect = (0, diff, ORIG_WIDTH, ORIG_HEIGHT - diff)

In [27]:
images = []
for pic_file in tqdm(os.listdir(PIC_DIR)[:IMAGES_COUNT]):
    pic = Image.open(PIC_DIR + pic_file).crop(crop_rect)
    pic.thumbnail((WIDTH, HEIGHT), Image.ANTIALIAS)
    images.append(np.uint8(pic))

100%|██████████| 10000/10000 [00:27<00:00, 369.17it/s]


In [28]:
images = np.array(images) / 255
print(images.shape)

(10000, 128, 128, 3)


In [29]:
from matplotlib import pyplot as plt